In [52]:
import pandas as pd
import numpy as np

In [101]:
d_ms = pd.read_csv('../example_data/clustering/0815.csv')

In [102]:
d_ms.head()

,Average Rt(min),Average Mz,S/N average,20180815_CEC_CAL-8-no4_MSpos_1,20180815_CEC_CAL-8-no4_MSpos_2,20180815_CEC_CAL-8-no4_MSpos_3,20180815_CEC_CAL-8-no4_MSpos_4,20180815_CEC_CAL-8-no4_MSpos_5,20180815_CEC_CAL-8-no4_MSpos_6,20180815_CEC_CAL-8-no4_MSpos_7,...,20180815_SR520-Creek_Mix6A_3,20180815_SR520-Creek_Mix6B_1,20180815_SR520-Creek_Mix6B_2,20180815_SR520-Creek_Mix6B_3,20180815_Swan-Creek-Dec_1,20180815_Swan-Creek-Dec_2,20180815_Swan-Creek-Dec_3,20180815_Swan-Creek-May_1,20180815_Swan-Creek-May_2,20180815_Swan-Creek-May_3
0,8.273,100.03867,40.44,0,0,88,0,0,89,0,...,238,113,184,202,0,0,0,0,76,0
1,8.304,100.03956,77.03,0,0,88,0,0,88,0,...,238,217,184,202,0,0,0,0,73,0
2,7.097,100.07603,52.53,64,231,104,247,259,145,294,...,1375,1139,511,1159,1201,1337,1723,573,2585,1585
3,5.226,100.11173,31.39,1629,882,1850,2217,1254,946,1095,...,18206,20583,20713,19942,21409,21764,23078,18056,18226,17470
4,4.944,100.11175,14.70,759,934,396,1676,583,639,336,...,10508,8889,14385,6274,12796,14584,13262,7576,12092,11060


In [104]:
d_ms = d_ms.rename(columns={'Average Rt(min)': 'Average RT (min)', 'Average Mz': 'Average m/z', 'S/N average': 'Average sn'})
d_ms.insert(3, "Average score", 1)

In [110]:
def d_clean(dataframe, rt_range=[0, 30], mz_range=[0, 1200], sn_thres=3, score_thres=0, area_thres=5000): #Update with *args or **args in future updates
    #Area thres update
    drop_index = np.argwhere(np.asarray(dataframe[dataframe.columns[4:]].max(axis=1)) < area_thres).reshape(1,-1)
    df_c = dataframe.drop(drop_index[0])
    
    df_c = df_c[(df_c['Average RT (min)'] > rt_range[0]) & (df_c['Average RT (min)'] < rt_range[1])]
    df_c = df_c[(df_c['Average m/z'] > mz_range[0]) & (df_c['Average m/z'] < mz_range[1])]
    df_c = df_c[df_c['Average sn'] >= sn_thres]
    df_c = df_c[df_c['Average score'] >= score_thres]
    df_c.reset_index(inplace=True)
    df_c.drop(columns=['index'],inplace=True)
    
    return df_c

In [111]:
df_c = d_clean(d_ms)

In [113]:
df_c.head()

,Average RT (min),Average m/z,Average sn,Average score,20180815_CEC_CAL-8-no4_MSpos_1,20180815_CEC_CAL-8-no4_MSpos_2,20180815_CEC_CAL-8-no4_MSpos_3,20180815_CEC_CAL-8-no4_MSpos_4,20180815_CEC_CAL-8-no4_MSpos_5,20180815_CEC_CAL-8-no4_MSpos_6,...,20180815_SR520-Creek_Mix6A_3,20180815_SR520-Creek_Mix6B_1,20180815_SR520-Creek_Mix6B_2,20180815_SR520-Creek_Mix6B_3,20180815_Swan-Creek-Dec_1,20180815_Swan-Creek-Dec_2,20180815_Swan-Creek-Dec_3,20180815_Swan-Creek-May_1,20180815_Swan-Creek-May_2,20180815_Swan-Creek-May_3
0,8.273,100.03867,40.44,1,0,0,88,0,0,89,...,238,113,184,202,0,0,0,0,76,0
1,8.304,100.03956,77.03,1,0,0,88,0,0,88,...,238,217,184,202,0,0,0,0,73,0
2,7.097,100.07603,52.53,1,64,231,104,247,259,145,...,1375,1139,511,1159,1201,1337,1723,573,2585,1585
3,5.226,100.11173,31.39,1,1629,882,1850,2217,1254,946,...,18206,20583,20713,19942,21409,21764,23078,18056,18226,17470
4,4.944,100.11175,14.70,1,759,934,396,1676,583,639,...,10508,8889,14385,6274,12796,14584,13262,7576,12092,11060
